In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import operator
import math
import random

from scipy.linalg import norm
from scipy.spatial.distance import cdist
from collections import defaultdict

from sklearn import datasets
from sklearn.datasets import make_blobs

In [47]:
# ignoring warnings
import warnings
warnings.simplefilter("ignore")
np.random.seed(42)

In [48]:
#Load Dataset
Home_data = pd.read_csv('HomeC.csv')
nan_removed = pd.DataFrame.dropna(pd.DataFrame(Home_data))
nan_removed

time  use [kW]  gen [kW]  House overall [kW]  Dishwasher [kW]  \
0       1451624400  0.932833  0.003483            0.932833         0.000033   
1       1451624401  0.934333  0.003467            0.934333         0.000000   
2       1451624402  0.931817  0.003467            0.931817         0.000017   
3       1451624403  1.022050  0.003483            1.022050         0.000017   
4       1451624404  1.139400  0.003467            1.139400         0.000133   
...            ...       ...       ...                 ...              ...   
503905  1452128305  1.601233  0.003183            1.601233         0.000050   
503906  1452128306  1.599333  0.003233            1.599333         0.000050   
503907  1452128307  1.924267  0.003217            1.924267         0.000033   
503908  1452128308  1.978200  0.003217            1.978200         0.000050   
503909  1452128309  1.990950  0.003233            1.990950         0.000050   

        Furnace 1 [kW]  Furnace 2 [kW]  Home office [kW]  Fridge [kW]  \
0             0.020700        0.061917          0.442633     0.124150   
1             0.020717        0.063817          0.444067     0.124000   
2             0.020700        0.062317          0.446067     0.123533   
3             0.106900        0.068517          0.446583     0.123133   
4             0.236933        0.063983          0.446533     0.122850   
...                ...             ...               ...          ...   
503905        0.085267        0.642417          0.041783     0.005267   
503906        0.104017        0.625033          0.041750     0.005233   
503907        0.422383        0.637733          0.042033     0.004983   
503908        0.495667        0.620367          0.042100     0.005333   
503909        0.494700        0.634133          0.042100     0.004917   

        Wine cellar [kW]  ...  visibility     summary  apparentTemperature  \
0               0.006983  ...       10.00       Clear                29.26   
1               0.006983  ...       10.00       Clear                29.26   
2               0.006983  ...       10.00       Clear                29.26   
3               0.006983  ...       10.00       Clear                29.26   
4               0.006850  ...       10.00       Clear                29.26   
...                  ...  ...         ...         ...                  ...   
503905          0.008667  ...        8.74  Light Rain                29.45   
503906          0.008433  ...        8.74  Light Rain                29.45   
503907          0.008467  ...        8.74  Light Rain                29.45   
503908          0.008233  ...        8.74  Light Rain                29.45   
503909          0.008133  ...        8.74  Light Rain                29.45   

        pressure  windSpeed  cloudCover  windBearing  precipIntensity  \
0        1016.91       9.18  cloudCover        282.0           0.0000   
1        1016.91       9.18  cloudCover        282.0           0.0000   
2        1016.91       9.18  cloudCover        282.0           0.0000   
3        1016.91       9.18  cloudCover        282.0           0.0000   
4        1016.91       9.18  cloudCover        282.0           0.0000   
...          ...        ...         ...          ...              ...   
503905   1011.49       6.72        0.31        186.0           0.0101   
503906   1011.49       6.72        0.31        186.0           0.0101   
503907   1011.49       6.72        0.31        186.0           0.0101   
503908   1011.49       6.72        0.31        186.0           0.0101   
503909   1011.49       6.72        0.31        186.0           0.0101   

        dewPoint  precipProbability  
0          24.40               0.00  
1          24.40               0.00  
2          24.40               0.00  
3          24.40               0.00  
4          24.40               0.00  
...          ...                ...  
503905     31.27               0.51  
503906     31.27               0.51  
503907     31.27               0

In [49]:
# df = pd.DataFrame(nan_removed)
df_selected = nan_removed[nan_removed['summary'].str.contains('Breezy', case = False).fillna(False) | nan_removed['summary'].str.contains('Flurries', case = False).fillna(False) | nan_removed['summary'].str.contains('overcast', case = False).fillna(False)] 
X = (df_selected[['temperature', 'apparentTemperature', 'windSpeed', 'windBearing']]).values
y = df_selected['use [kW]'].values
data = X
X

array([[ 37.14,  30.36,   9.45, 273.  ],
       [ 37.14,  30.36,   9.45, 273.  ],
       [ 37.14,  30.36,   9.45, 273.  ],
       ...,
       [ 36.63,  29.03,  10.97, 187.  ],
       [ 36.63,  29.03,  10.97, 187.  ],
       [ 36.63,  29.03,  10.97, 187.  ]])

In [50]:

# Mendefinisikan fungsi jarak euclidean
def euclidean_distance(point1,point2):
    dis=0
    for i in range(len(point1)):
        dis+=(point1[i]-point2[i])**2
    return dis**0.5

# Initializing the Membership Matrix
def initialize_membership_matrix(n,c):
    member_mat=list()
    for i in range(n):
        random_list=[random.random() for x in range(c)]
        summation=sum(random_list)
        for i in range(len(random_list)):
            random_list[i]=random_list[i]/summation
        member_mat.append(random_list)
    return member_mat

def update_centroid(member_mat):
    centroids={}
    for j in range(c):
        temp=[]
        for k in range(p):      
            add=0
            for i in range(n):
                add+=member_mat[i][j]**m
            x=0
            for i in range(n):
                x+=(member_mat[i][j]**m)*(data[i][k])
            val=x/add
            temp.append(val)
        centroids[j]=temp
    return centroids

def update_membership_matrix(member_mat,centroids):
    ratio=float(2/(m-1))
    for i in range(n):
        distances=list()
        for j in range(c):
            distances.append(euclidean_distance(data[i],centroids[j]))
        for j in range(c):
            den = sum([math.pow(float(distances[j]/distances[q]), ratio) for q in range(c)])
            member_mat[i][j] = float(1/den)         
            
    return member_mat
        
def find_cluster(member_mat):
    clusters=list()
    for i in range(n):
        max_val, idx = max((val, idx) for (idx, val) in enumerate(member_mat[i]))
        clusters.append(idx)
    return clusters

def check(old_member_mat,member_mat):
    diff=0
    for i in range(n):
        for j in range(c):
            diff+=old_member_mat[i][j]-member_mat[i][j]
    if(diff<0.01):
        return True
    return False

def fuzzy_c_mean(max_iter):
    member_mat=initialize_membership_matrix(n,c)
    for i in range(max_iter):
        centroids=update_centroid(member_mat)
        old_member_mat=member_mat
        member_mat=update_membership_matrix(member_mat,centroids)
        cluster=find_cluster(member_mat)
        if(check(old_member_mat,member_mat))<0.01:
            print(i)
            break    
    
    return cluster,centroids

# labeling the clusters
def label_clusters(clusters):
    z=0
    o=0
    t=0
    dict=defaultdict(int)

    for i in range(50):
        if(clusters[i]==0):
            z=z+1
        elif(clusters[i]==1):
            o=o+1
        else:
            t=t+1
    dict[z]=0
    dict[o]=1
    dict[t]=2
    classes=[]
    fin1=max(z,max(o,t))
    
    classes.append(dict[fin1])
    z=0
    o=0
    t=0
    for i in range(50,100):
        if(clusters[i]==0):
            z=z+1
        elif(clusters[i]==1):
            o=o+1
        else:
            t=t+1
    dict[z]=0
    dict[o]=1
    dict[t]=2

    fin1=max(z,max(o,t))
    
    classes.append(dict[fin1])
    z=0
    o=0
    t=0
    for i in range(100,150):
        if(clusters[i]==0):
            z=z+1
        elif(clusters[i]==1):
            o=o+1
        else:
            t=t+1
    dict[z]=0
    dict[o]=1
    dict[t]=2
    fin1=max(z,max(o,t))
    classes.append(dict[fin1])
    
    return classes

def confusion_matrix(clusters,classes,labels):
    c_matrix=[[0 for i in range(labels)] for j in range(labels)]
    m=0
    n=0
    q=0
    for i in range(50):
        if clusters[i]==classes[0]:
            m=m+1
        elif clusters[i]==classes[1]:
            n=n+1
        else:
            q=q+1
    c_matrix[0][0]=m
    c_matrix[0][1]=n
    c_matrix[0][2]=q
    m=0
    n=0
    q=0
    for i in range(50,100):
        if clusters[i]==classes[0]:
            m=m+1
        elif clusters[i]==classes[1]:
            n=n+1
        else:
            q=q+1
    c_matrix[1][0]=m
    c_matrix[1][1]=n
    c_matrix[1][2]=q
    m=0
    n=0
    q=0

    for i in range(100,150):
        if clusters[i]==classes[0]:
            m=m+1
        elif clusters[i]==classes[1]:
            n=n+1
        else:
            q=q+1
    c_matrix[2][0]=m
    c_matrix[2][1]=n
    c_matrix[2][2]=q
    
    return c_matrix

# Performance Metric
class Metrics:
    def __init__(self,confusion_m):
        self.confusion_m=confusion_m
        self.total=np.sum(confusion_m)
        self.diagonal=np.sum(np.diag(confusion_m))
    
    def accuracy(self):
        accuracy=(self.diagonal/self.total)
        return accuracy
    
    def recall(self):
        recall=np.diag(self.confusion_m)/np.sum(self.confusion_m,axis=1)
        recall=np.mean(recall)
        return recall
    
    def precision(self):
        precision=np.diag(self.confusion_m)/np.sum(self.confusion_m,axis=0)
        precision=np.mean(precision)
        return precision
    
    def f1_score(self,precision,recall):
        f1_score=(2*precision*recall)/(precision+recall)
        return f1_score

In [51]:
# utility values
m = 2
n = len(data)
c = 3
p = len(data[0])
max_iter = 100

# Prediksi menggunakan FCM Clustering
clusters,centroids=fuzzy_c_mean(max_iter)
print("Final Centroid points are:")
print(centroids)

classes=label_clusters(clusters)

Final Centroid points are:
{0: [41.8479212301012, 39.33329409909101, 6.88352569920408, 185.14517978700894], 1: [43.445984329228665, 40.41202108813998, 6.340924568782547, 36.67745733273945], 2: [47.13568472488469, 42.410850544509934, 16.325221947172903, 300.69085276593137]}


In [52]:
# Performance
matrix=confusion_matrix(clusters,classes,3)
performance=Metrics(matrix)

accuracy=performance.accuracy()
recall=performance.recall()
precision=performance.precision()
f1_score=performance.f1_score(precision,recall)

print('confusion matrix is:',end='\n')
print(np.array(matrix),end='\n')

print("Accuracy of the model is {}".format(accuracy*100))
print("Recall of the model is {}".format(recall*100))
print("Precision of the model is {}".format(precision*100))
print("F1-Score of the model is {}".format(f1_score*100))

confusion matrix is:
[[50  0  0]
 [ 8 42  0]
 [ 0 16 34]]
Accuracy of the model is 84.0
Recall of the model is 84.0
Precision of the model is 86.20689655172414
F1-Score of the model is 85.08914100486223
